<a href="https://colab.research.google.com/github/sungkim11/TaxGPT/blob/main/Create_IRSIRB_DB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install Python Packages

In [ ]:
%%writefile requirements.txt
openai
chromadb
langchain
tiktoken
unstructured

Overwriting requirements.txt


In [ ]:
%pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Import Python Packages

In [ ]:
import os
import platform

import openai
import chromadb
import langchain

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import TokenTextSplitter
from langchain.llms import OpenAI
from langchain.document_loaders import UnstructuredURLLoader

import urllib
from urllib import request

print('Python: ', platform.python_version())

Python:  3.9.16


Mount Storage - Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Set OpenAI API Key

In [ ]:
os.environ["OPENAI_API_KEY"] = "openai api key"

List Interanl Revenue Bulletins URLs

In [ ]:
urls = [
    "https://www.irs.gov/irb/2005-38_IRB",
    "https://www.irs.gov/irb/2009-30_IRB",
    "https://www.irs.gov/irb/2017-23_IRB",
    "https://www.irs.gov/irb/2013-07_IRB",
    "https://www.irs.gov/irb/2006-52_IRB",
    "https://www.irs.gov/irb/2004-45_irb",
    "https://www.irs.gov/irb/2005-31_IRB",
    "https://www.irs.gov/irb/2021-44_IRB",
    "https://www.irs.gov/irb/2011-37_IRB",
    "https://www.irs.gov/irb/2018-38_IRB",
    "https://www.irs.gov/irb/2005-48_IRB",
    "https://www.irs.gov/irb/2015-52_IRB",
    "https://www.irs.gov/irb/2018-47_IRB",
    "https://www.irs.gov/irb/2007-39_IRB",
    "https://www.irs.gov/irb/2004-13_IRB",
    "https://www.irs.gov/irb/2018-06_IRB",
    "https://www.irs.gov/irb/2016-46_IRB",
    "https://www.irs.gov/irb/2019-05_IRB",
    "https://www.irs.gov/irb/2011-32_IRB",
    "https://www.irs.gov/irb/2017-11_IRB",
    "https://www.irs.gov/irb/2007-49_IRB",
    "https://www.irs.gov/irb/2011-44_IRB",
    "https://www.irs.gov/irb/2004-17_IRB",
    "https://www.irs.gov/irb/2007-40_IRB",
    "https://www.irs.gov/irb/2005-21_IRB",
    "https://www.irs.gov/irb/2004-44_IRB",
    "https://www.irs.gov/irb/2004-24_IRB",
    "https://www.irs.gov/irb/2015-22_IRB",
    "https://www.irs.gov/irb/2005-52_IRB",
    "https://www.irs.gov/irb/2007-23_IRB",
    "https://www.irs.gov/irb/2011-45_IRB",
    "https://www.irs.gov/irb/2022-05_IRB",
    "https://www.irs.gov/irb/2016-06_IRB",
    "https://www.irs.gov/irb/2013-13_IRB",
    "https://www.irs.gov/irb/2003-28_IRB",
    "https://www.irs.gov/irb/2020-52_IRB",
    "https://www.irs.gov/irb/2008-29_IRB",
    "https://www.irs.gov/irb/2006-46_IRB",
    "https://www.irs.gov/irb/2003-42_IRB",
    "https://www.irs.gov/irb/2014-15_IRB",
    "https://www.irs.gov/irb/2008-31_IRB",
    "https://www.irs.gov/irb/2005-17_IRB",
    "https://www.irs.gov/irb/2008-12_IRB",
    "https://www.irs.gov/irb/2020-34_IRB",
    "https://www.irs.gov/irb/2014-14_IRB",
    "https://www.irs.gov/irb/2011-31_IRB",
    "https://www.irs.gov/irb/2011-24_IRB",
    "https://www.irs.gov/irb/2016-34_IRB",
    "https://www.irs.gov/irb/2004-39_IRB",
    "https://www.irs.gov/irb/2020-41_IRB",
    "https://www.irs.gov/irb/2018-32_IRB",
    "https://www.irs.gov/irb/2011-10_IRB",
    "https://www.irs.gov/irb/2010-27_IRB",
    "https://www.irs.gov/irb/2020-48_IRB",
    "https://www.irs.gov/irb/2018-17_IRB",
    "https://www.irs.gov/irb/2006-04_IRB",
    "https://www.irs.gov/irb/2022-04_IRB",
    "https://www.irs.gov/irb/2011-51_IRB",
    "https://www.irs.gov/irb/2016-29_IRB",
    "https://www.irs.gov/irb/2004-06_IRB",
    "https://www.irs.gov/irb/2005-49_IRB",
    "https://www.irs.gov/irb/2020-22_IRB",
    "https://www.irs.gov/irb/2017-34_IRB",
    "https://www.irs.gov/irb/2022-31_IRB",
    "https://www.irs.gov/irb/2022-13_IRB",
    "https://www.irs.gov/irb/2017-27_IRB",
    "https://www.irs.gov/irb/2022-41_IRB",
    "https://www.irs.gov/irb/2013-09_IRB",
    "https://www.irs.gov/irb/2004-50_IRB",
    "https://www.irs.gov/irb/2020-43_IRB",
    "https://www.irs.gov/irb/2019-28_IRB",
    "https://www.irs.gov/irb/2011-14_IRB",
    "https://www.irs.gov/irb/2021-12_IRB",
    "https://www.irs.gov/irb/2016-03_IRB",
    "https://www.irs.gov/irb/2007-16_IRB",
    "https://www.irs.gov/irb/2015-38_IRB",
    "https://www.irs.gov/irb/2013-48_IRB",
    "https://www.irs.gov/irb/2005-15_IRB",
    "https://www.irs.gov/irb/2017-17_IRB",
    "https://www.irs.gov/irb/2016-40_IRB",
    "https://www.irs.gov/irb/2015-13_IRB",
    "https://www.irs.gov/irb/2011-09_IRB",
    "https://www.irs.gov/irb/2013-49_IRB",
    "https://www.irs.gov/irb/2009-11_IRB",
    "https://www.irs.gov/irb/2011-26_IRB",
    "https://www.irs.gov/irb/2008-20_IRB",
    "https://www.irs.gov/irb/2007-11_IRB",
    "https://www.irs.gov/irb/2023-10_IRB",
    "https://www.irs.gov/irb/2016-48_IRB",
    "https://www.irs.gov/irb/2003-44_IRB",
    "https://www.irs.gov/irb/2021-14_IRB",
    "https://www.irs.gov/irb/2012-36_IRB",
    "https://www.irs.gov/irb/2012-31_IRB",
    "https://www.irs.gov/irb/2003-36_IRB",
    "https://www.irs.gov/irb/2008-19_IRB",
    "https://www.irs.gov/irb/2014-37_IRB",
    "https://www.irs.gov/irb/2014-20_IRB",
    "https://www.irs.gov/irb/2020-40_IRB",
    "https://www.irs.gov/irb/2009-07_IRB",
    "https://www.irs.gov/irb/2006-31_IRB",
    "https://www.irs.gov/irb/2011-48_IRB",
    "https://www.irs.gov/irb/2015-40_IRB",
    "https://www.irs.gov/irb/2012-19_IRB",
    "https://www.irs.gov/irb/2012-09_IRB",
    "https://www.irs.gov/irb/2022-39_IRB",
    "https://www.irs.gov/irb/2009-28_IRB",
    "https://www.irs.gov/irb/2013-02_IRB",
    "https://www.irs.gov/irb/2014-12_IRB",
    "https://www.irs.gov/irb/2018-26_IRB",
    "https://www.irs.gov/irb/2013-29_IRB",
    "https://www.irs.gov/irb/2006-44_IRB",
    "https://www.irs.gov/irb/2003-32_IRB",
    "https://www.irs.gov/irb/2016-45_IRB",
    "https://www.irs.gov/irb/2019-01_IRB",
    "https://www.irs.gov/irb/2017-41_IRB",
    "https://www.irs.gov/irb/2021-29_IRB",
    "https://www.irs.gov/irb/2016-18_IRB",
    "https://www.irs.gov/irb/2012-01_IRB",
    "https://www.irs.gov/irb/2019-15_IRB",
    "https://www.irs.gov/irb/2012-17_IRB",
    "https://www.irs.gov/irb/2011-35_IRB",
    "https://www.irs.gov/irb/2007-06_IRB",
    "https://www.irs.gov/irb/2022-18_IRB",
    "https://www.irs.gov/irb/2019-47_IRB",
    "https://www.irs.gov/irb/2010-11_IRB",
    "https://www.irs.gov/irb/2009-14_IRB",
    "https://www.irs.gov/irb/2015-32_IRB",
    "https://www.irs.gov/irb/2019-13_IRB",
    "https://www.irs.gov/irb/2005-16_IRB",
    "https://www.irs.gov/irb/2010-04_IRB",
    "https://www.irs.gov/irb/2014-29_IRB",
    "https://www.irs.gov/irb/2004-49_IRB",
    "https://www.irs.gov/irb/2005-45_IRB",
    "https://www.irs.gov/irb/2006-09_IRB",
    "https://www.irs.gov/irb/2006-50_IRB",
    "https://www.irs.gov/irb/2005-46_IRB",
    "https://www.irs.gov/irb/2004-31_IRB",
    "https://www.irs.gov/irb/2021-26_IRB",
    "https://www.irs.gov/irb/2007-02_IRB",
    "https://www.irs.gov/irb/2008-01_IRB",
    "https://www.irs.gov/irb/2008-14_IRB",
    "https://www.irs.gov/irb/2018-03_IRB",
    "https://www.irs.gov/irb/2006-14_IRB",
    "https://www.irs.gov/irb/2013-04_IRB",
    "https://www.irs.gov/irb/2023-04_IRB",
    "https://www.irs.gov/irb/2008-02_IRB",
    "https://www.irs.gov/irb/2010-16_IRB",
    "https://www.irs.gov/irb/2008-44_IRB",
    "https://www.irs.gov/irb/2004-28_IRB",
    "https://www.irs.gov/irb/2008-52_IRB",
    "https://www.irs.gov/irb/2005-36_IRB",
    "https://www.irs.gov/irb/2007-47_IRB",
    "https://www.irs.gov/irb/2021-27_IRB",
    "https://www.irs.gov/irb/2012-45_IRB",
    "https://www.irs.gov/irb/2014-45_IRB",
    "https://www.irs.gov/irb/2009-25_IRB",
    "https://www.irs.gov/irb/2003-41_IRB",
    "https://www.irs.gov/irb/2020-08_IRB",
    "https://www.irs.gov/irb/2004-47_IRB",
    "https://www.irs.gov/irb/2004-14_IRB",
    "https://www.irs.gov/irb/2016-20_IRB",
    "https://www.irs.gov/irb/2011-47_IRB",
    "https://www.irs.gov/irb/2008-07_IRB",
    "https://www.irs.gov/irb/2007-45_IRB",
    "https://www.irs.gov/irb/2004-25_IRB",
    "https://www.irs.gov/irb/2017-16_IRB",
    "https://www.irs.gov/irb/2011-20_IRB",
    "https://www.irs.gov/irb/2006-29_IRB",
    "https://www.irs.gov/irb/2006-49_IRB",
    "https://www.irs.gov/irb/2013-40_IRB",
    "https://www.irs.gov/irb/2016-14_IRB",
    "https://www.irs.gov/irb/2005-39_IRB",
    "https://www.irs.gov/irb/2017-19_IRB",
    "https://www.irs.gov/irb/2007-42_IRB",
    "https://www.irs.gov/irb/2012-52_IRB",
    "https://www.irs.gov/irb/2008-04_IRB",
    "https://www.irs.gov/irb/2005-50_IRB",
    "https://www.irs.gov/irb/2008-08_IRB",
    "https://www.irs.gov/irb/2023-07_IRB",
    "https://www.irs.gov/irb/2020-12_IRB",
    "https://www.irs.gov/irb/2017-35_IRB",
    "https://www.irs.gov/irb/2009-51_IRB",
    "https://www.irs.gov/irb/2006-23_IRB",
    "https://www.irs.gov/irb/2011-29_IRB",
    "https://www.irs.gov/irb/2015-02_IRB",
    "https://www.irs.gov/irb/2008-11_IRB",
    "https://www.irs.gov/irb/2022-25_IRB",
    "https://www.irs.gov/irb/2017-52_IRB",
    "https://www.irs.gov/irb/2012-04_IRB",
    "https://www.irs.gov/irb/2021-15_IRB",
    "https://www.irs.gov/irb/2009-16_irb",
    "https://www.irs.gov/irb/2021-21_IRB",
    "https://www.irs.gov/irb/2018-52_IRB",
    "https://www.irs.gov/irb/2005-02_IRB",
    "https://www.irs.gov/irb/2008-38_IRB",
    "https://www.irs.gov/irb/2010-19_IRB",
    "https://www.irs.gov/irb/2012-24_IRB",
    "https://www.irs.gov/irb/2018-16_IRB",
    "https://www.irs.gov/irb/2007-52_IRB",
    "https://www.irs.gov/irb/2021-34_IRB",
    "https://www.irs.gov/irb/2018-44_IRB",
    "https://www.irs.gov/irb/2014-39_IRB",
    "https://www.irs.gov/irb/2009-38_IRB",
    "https://www.irs.gov/irb/2005-32_IRB",
    "https://www.irs.gov/irb/2004-51_IRB",
    "https://www.irs.gov/irb/2013-23_IRB",
    "https://www.irs.gov/irb/2012-28_IRB",
    "https://www.irs.gov/irb/2008-32_IRB",
    "https://www.irs.gov/irb/2017-38_IRB",
    "https://www.irs.gov/irb/2022-30_IRB",
    "https://www.irs.gov/irb/2016-39_IRB",
    "https://www.irs.gov/irb/2018-07_IRB",
    "https://www.irs.gov/irb/2007-27_IRB",
    "https://www.irs.gov/irb/2014-19_IRB",
    "https://www.irs.gov/irb/2014-22_IRB",
    "https://www.irs.gov/irb/2007-24_IRB",
    "https://www.irs.gov/irb/2010-36_IRB",
    "https://www.irs.gov/irb/2017-21_IRB",
    "https://www.irs.gov/irb/2012-03_IRB",
    "https://www.irs.gov/irb/2014-38_IRB",
    "https://www.irs.gov/irb/2019-45_IRB",
    "https://www.irs.gov/irb/2019-02_IRB",
    "https://www.irs.gov/irb/2009-33_IRB",
    "https://www.irs.gov/irb/2005-12_IRB",
    "https://www.irs.gov/irb/2014-47_IRB",
    "https://www.irs.gov/irb/2008-03_IRB",
    "https://www.irs.gov/irb/2012-05_IRB",
    "https://www.irs.gov/irb/2004-41_IRB",
    "https://www.irs.gov/irb/2021-52_IRB",
    "https://www.irs.gov/irb/2012-12_IRB",
    "https://www.irs.gov/irb/2014-13_IRB",
    "https://www.irs.gov/irb/2020-27_IRB",
    "https://www.irs.gov/irb/2012-27_IRB",
    "https://www.irs.gov/irb/2020-03_IRB",
    "https://www.irs.gov/irb/2021-03_IRB",
    "https://www.irs.gov/irb/2006-38_IRB",
    "https://www.irs.gov/irb/2019-49_IRB",
    "https://www.irs.gov/irb/2012-42_IRB",
    "https://www.irs.gov/irb/2008-45_IRB",
    "https://www.irs.gov/irb/2005-40_IRB",
    "https://www.irs.gov/irb/2014-06_IRB",
    "https://www.irs.gov/irb/2020-37_IRB",
    "https://www.irs.gov/irb/2020-18_IRB",
    "https://www.irs.gov/irb/2019-19_IRB",
    "https://www.irs.gov/irb/2012-14_IRB",
    "https://www.irs.gov/irb/2011-02_IRB",
    "https://www.irs.gov/irb/2014-24_IRB",
    "https://www.irs.gov/irb/2017-01_IRB",
    "https://www.irs.gov/irb/2021-20_IRB",
    "https://www.irs.gov/irb/2017-51_IRB",
    "https://www.irs.gov/irb/2012-08_IRB",
    "https://www.irs.gov/irb/2008-05_IRB",
    "https://www.irs.gov/irb/2018-09_IRB",
    "https://www.irs.gov/irb/2022-19_IRB",
    "https://www.irs.gov/irb/2018-49_IRB",
    "https://www.irs.gov/irb/2009-29_IRB",
    "https://www.irs.gov/irb/2005-24_IRB",
    "https://www.irs.gov/irb/2016-22_IRB",
    "https://www.irs.gov/irb/2014-18_IRB",
    "https://www.irs.gov/irb/2009-26_IRB",
    "https://www.irs.gov/irb/2006-16_IRB",
    "https://www.irs.gov/irb/2015-19_IRB",
    "https://www.irs.gov/irb/2005-03_IRB",
    "https://www.irs.gov/irb/2022-21_IRB",
    "https://www.irs.gov/irb/2009-23_IRB",
    "https://www.irs.gov/irb/2015-31_IRB",
    "https://www.irs.gov/irb/2016-41_IRB",
    "https://www.irs.gov/irb/2020-04_IRB",
    "https://www.irs.gov/irb/2012-41_IRB",
    "https://www.irs.gov/irb/2015-26_IRB",
    "https://www.irs.gov/irb/2022-03_IRB",
    "https://www.irs.gov/irb/2022-36_IRB",
    "https://www.irs.gov/irb/2022-12_IRB",
    "https://www.irs.gov/irb/2021-43_IRB",
    "https://www.irs.gov/irb/2018-21_IRB",
    "https://www.irs.gov/irb/2018-22_IRB",
    "https://www.irs.gov/irb/2017-20_IRB",
    "https://www.irs.gov/irb/2015-34_IRB",
    "https://www.irs.gov/irb/2015-08_IRB",
    "https://www.irs.gov/irb/2017-15_IRB",
    "https://www.irs.gov/irb/2017-14_IRB",
    "https://www.irs.gov/irb/2021-24_IRB",
    "https://www.irs.gov/irb/2015-18_IRB",
    "https://www.irs.gov/irb/2010-21_IRB",
    "https://www.irs.gov/irb/2021-23_IRB",
    "https://www.irs.gov/irb/2011-52_IRB",
    "https://www.irs.gov/irb/2009-41_IRB",
    "https://www.irs.gov/irb/2009-01_IRB",
    "https://www.irs.gov/irb/2013-37_IRB",
    "https://www.irs.gov/irb/2009-48_IRB",
    "https://www.irs.gov/irb/2007-34_IRB",
    "https://www.irs.gov/irb/2017-04_IRB",
    "https://www.irs.gov/irb/2004-43_IRB",
    "https://www.irs.gov/irb/2021-51_IRB",
    "https://www.irs.gov/irb/2004-37_IRB",
    "https://www.irs.gov/irb/2005-28_IRB",
    "https://www.irs.gov/irb/2004-02_IRB",
    "https://www.irs.gov/irb/2003-33_IRB",
    "https://www.irs.gov/irb/2019-46_IRB",
    "https://www.irs.gov/irb/2022-32_IRB",
    "https://www.irs.gov/irb/2008-41_IRB",
    "https://www.irs.gov/irb/2020-50_IRB",
    "https://www.irs.gov/irb/2012-34_IRB",
    "https://www.irs.gov/irb/2016-52_IRB",
    "https://www.irs.gov/irb/2007-38_IRB",
    "https://www.irs.gov/irb/2009-43_IRB",
    "https://www.irs.gov/irb/2015-42_IRB",
    "https://www.irs.gov/irb/2011-11_IRB",
    "https://www.irs.gov/irb/2019-24_IRB",
    "https://www.irs.gov/irb/2014-50_IRB",
    "https://www.irs.gov/irb/2019-42_IRB",
    "https://www.irs.gov/irb/2013-24_IRB",
    "https://www.irs.gov/irb/2013-32_IRB",
    "https://www.irs.gov/irb/2011-08_IRB",
    "https://www.irs.gov/irb/2010-41_IRB",
    "https://www.irs.gov/irb/2010-12_IRB",
    "https://www.irs.gov/irb/2009-19_IRB",
    "https://www.irs.gov/irb/2015-28_IRB",
    "https://www.irs.gov/irb/2019-11_IRB",
    "https://www.irs.gov/irb/2009-32_IRB",
    "https://www.irs.gov/irb/2007-48_IRB",
    "https://www.irs.gov/irb/2007-26_IRB",
    "https://www.irs.gov/irb/2005-44_IRB",
    "https://www.irs.gov/irb/2008-47_IRB",
    "https://www.irs.gov/irb/2022-45_IRB",
    "https://www.irs.gov/irb/2017-37_IRB",
    "https://www.irs.gov/irb/2014-40_IRB",
    "https://www.irs.gov/irb/2020-09_IRB",
    "https://www.irs.gov/irb/2007-01_IRB",
    "https://www.irs.gov/irb/2006-11_IRB",
    "https://www.irs.gov/irb/2004-35_IRB",
    "https://www.irs.gov/irb/2015-03_IRB",
    "https://www.irs.gov/irb/2013-44_IRB",
    "https://www.irs.gov/irb/2022-29_IRB",
    "https://www.irs.gov/irb/2019-34_IRB",
    "https://www.irs.gov/irb/2019-25_IRB",
    "https://www.irs.gov/irb/2016-10_IRB",
    "https://www.irs.gov/irb/2020-53_IRB",
    "https://www.irs.gov/irb/2016-37_IRB",
    "https://www.irs.gov/irb/2006-05_IRB",
    "https://www.irs.gov/irb/2021-49_IRB",
    "https://www.irs.gov/irb/2004-15_IRB",
    "https://www.irs.gov/irb/2014-01_IRB",
    "https://www.irs.gov/irb/2018-01_IRB",
    "https://www.irs.gov/irb/2005-01_IRB",
    "https://www.irs.gov/irb/2015-46_IRB",
    "https://www.irs.gov/irb/2010-45_IRB",
    "https://www.irs.gov/irb/2006-32_IRB",
    "https://www.irs.gov/irb/2021-13_IRB",
    "https://www.irs.gov/irb/2018-24_IRB",
    "https://www.irs.gov/irb/2023-12_IRB",
    "https://www.irs.gov/irb/2004-30_IRB",
    "https://www.irs.gov/irb/2006-22_IRB",
    "https://www.irs.gov/irb/2013-30_IRB",
    "https://www.irs.gov/irb/2012-50_IRB",
    "https://www.irs.gov/irb/2018-10_IRB",
    "https://www.irs.gov/irb/2018-50_IRB",
    "https://www.irs.gov/irb/2007-05_IRB",
    "https://www.irs.gov/irb/2006-33_IRB",
    "https://www.irs.gov/irb/2007-33_IRB",
    "https://www.irs.gov/irb/2016-16_IRB",
    "https://www.irs.gov/irb/2022-28_IRB",
    "https://www.irs.gov/irb/2021-02_IRB",
    "https://www.irs.gov/irb/2020-13_IRB",
    "https://www.irs.gov/irb/2021-10_IRB",
    "https://www.irs.gov/irb/2006-27_IRB",
    "https://www.irs.gov/irb/2009-36_IRB",
    "https://www.irs.gov/irb/2017-31_IRB",
    "https://www.irs.gov/irb/2011-17_IRB",
    "https://www.irs.gov/irb/2003-29_IRB",
    "https://www.irs.gov/irb/2017-40_IRB",
    "https://www.irs.gov/irb/2018-18_IRB",
    "https://www.irs.gov/irb/2006-06_IRB",
    "https://www.irs.gov/irb/2005-51_IRB",
    "https://www.irs.gov/irb/2020-31_IRB",
    "https://www.irs.gov/irb/2013-38_IRB",
    "https://www.irs.gov/irb/2008-30_IRB",
    "https://www.irs.gov/irb/2005-10_IRB",
    "https://www.irs.gov/irb/2003-43_IRB",
    "https://www.irs.gov/irb/2010-15_IRB",
    "https://www.irs.gov/irb/2010-40_IRB",
    "https://www.irs.gov/irb/2013-43_IRB",
    "https://www.irs.gov/irb/2012-44_IRB",
    "https://www.irs.gov/irb/2013-16_IRB",
    "https://www.irs.gov/irb/2011-18_IRB",
    "https://www.irs.gov/irb/2010-25_IRB",
    "https://www.irs.gov/irb/2015-41_IRB",
    "https://www.irs.gov/irb/2011-06_IRB",
    "https://www.irs.gov/irb/2004-34_IRB",
    "https://www.irs.gov/irb/2018-25_IRB",
    "https://www.irs.gov/irb/2011-34_IRB",
    "https://www.irs.gov/irb/2016-21_IRB",
    "https://www.irs.gov/irb/2009-09_IRB",
    "https://www.irs.gov/irb/2022-01_IRB",
    "https://www.irs.gov/irb/2015-35_IRB",
    "https://www.irs.gov/irb/2006-03_IRB",
    "https://www.irs.gov/irb/2022-44_IRB",
    "https://www.irs.gov/irb/2010-18_IRB",
    "https://www.irs.gov/irb/2007-46_IRB",
    "https://www.irs.gov/irb/2012-10_IRB",
    "https://www.irs.gov/irb/2007-28_IRB",
    "https://www.irs.gov/irb/2021-04_IRB",
    "https://www.irs.gov/irb/2017-33_IRB",
    "https://www.irs.gov/irb/2005-06_IRB",
    "https://www.irs.gov/irb/2015-07_IRB",
    "https://www.irs.gov/irb/2018-14_IRB",
    "https://www.irs.gov/irb/2023-13_IRB",
    "https://www.irs.gov/irb/2014-35_IRB",
    "https://www.irs.gov/irb/2017-28_IRB",
    "https://www.irs.gov/irb/2009-45_IRB",
    "https://www.irs.gov/irb/2023-11_IRB",
    "https://www.irs.gov/irb/2013-35_IRB",
    "https://www.irs.gov/irb/2007-51_IRB",
    "https://www.irs.gov/irb/2022-11_IRB",
    "https://www.irs.gov/irb/2012-25_IRB",
    "https://www.irs.gov/irb/2004-48_IRB",
    "https://www.irs.gov/irb/2015-44_IRB",
    "https://www.irs.gov/irb/2015-51_IRB",
    "https://www.irs.gov/irb/2011-46_IRB",
    "https://www.irs.gov/irb/2007-30_IRB",
    "https://www.irs.gov/irb/2011-12_IRB",
    "https://www.irs.gov/irb/2008-49_IRB",
    "https://www.irs.gov/irb/2006-02_IRB",
    "https://www.irs.gov/irb/2022-20_IRB",
    "https://www.irs.gov/irb/2005-23_IRB",
    "https://www.irs.gov/irb/2022-38_IRB",
    "https://www.irs.gov/irb/2004-23_IRB",
    "https://www.irs.gov/irb/2021-22_IRB",
    "https://www.irs.gov/irb/2003-39_IRB",
    "https://www.irs.gov/irb/2003-30_IRB",
    "https://www.irs.gov/irb/2014-41_IRB",
    "https://www.irs.gov/irb/2020-16_IRB",
    "https://www.irs.gov/irb/2011-38_IRB",
    "https://www.irs.gov/irb/2017-32_IRB",
    "https://www.irs.gov/irb/2012-37_IRB",
    "https://www.irs.gov/irb/2009-17_IRB",
    "https://www.irs.gov/irb/2006-25_IRB",
    "https://www.irs.gov/irb/2008-24_IRB",
    "https://www.irs.gov/irb/2020-49_IRB",
    "https://www.irs.gov/irb/2018-08_IRB",
    "https://www.irs.gov/irb/2020-01_IRB",
    "https://www.irs.gov/irb/2013-27_IRB",
    "https://www.irs.gov/irb/2006-41_IRB",
    "https://www.irs.gov/irb/2010-06_IRB",
    "https://www.irs.gov/irb/2021-36_IRB",
    "https://www.irs.gov/irb/2021-11_IRB",
    "https://www.irs.gov/irb/2021-46_IRB",
    "https://www.irs.gov/irb/2012-48_IRB",
    "https://www.irs.gov/irb/2013-05_IRB",
    "https://www.irs.gov/irb/2022-47_IRB",
    "https://www.irs.gov/irb/2005-05_IRB",
    "https://www.irs.gov/irb/2021-37_IRB",
    "https://www.irs.gov/irb/2018-34_IRB",
    "https://www.irs.gov/irb/2014-09_IRB",
    "https://www.irs.gov/irb/2011-03_IRB",
    "https://www.irs.gov/irb/2016-36_IRB",
    "https://www.irs.gov/irb/2005-29_IRB",
    "https://www.irs.gov/irb/2015-48_IRB",
    "https://www.irs.gov/irb/2020-39_IRB",
    "https://www.irs.gov/irb/2020-28_IRB",
    "https://www.irs.gov/irb/2016-13_IRB",
    "https://www.irs.gov/irb/2020-24_IRB",
    "https://www.irs.gov/irb/2017-08_IRB",
    "https://www.irs.gov/irb/2015-12_IRB",
    "https://www.irs.gov/irb/2008-42_IRB",
    "https://www.irs.gov/irb/2022-34_IRB",
    "https://www.irs.gov/irb/2019-17_IRB",
    "https://www.irs.gov/irb/2021-25_IRB",
    "https://www.irs.gov/irb/2015-29_IRB",
    "https://www.irs.gov/irb/2016-28_IRB",
    "https://www.irs.gov/irb/2014-28_IRB",
    "https://www.irs.gov/irb/2011-16_IRB",
    "https://www.irs.gov/irb/2019-21_IRB",
    "https://www.irs.gov/irb/2004-42_IRB",
    "https://www.irs.gov/irb/2018-13_IRB",
    "https://www.irs.gov/irb/2011-05_IRB",
    "https://www.irs.gov/irb/2016-05_IRB",
    "https://www.irs.gov/irb/2013-08_IRB",
    "https://www.irs.gov/irb/2015-43_IRB",
    "https://www.irs.gov/irb/2022-46_IRB",
    "https://www.irs.gov/irb/2020-06_IRB",
    "https://www.irs.gov/irb/2019-33_IRB",
    "https://www.irs.gov/irb/2015-37_IRB",
    "https://www.irs.gov/irb/2013-31_IRB",
    "https://www.irs.gov/irb/2014-51_IRB",
    "https://www.irs.gov/irb/2011-01_IRB",
    "https://www.irs.gov/irb/2015-30_IRB",
    "https://www.irs.gov/irb/2015-49_IRB",
    "https://www.irs.gov/irb/2009-02_IRB",
    "https://www.irs.gov/irb/2008-34_IRB",
    "https://www.irs.gov/irb/2006-18_IRB",
    "https://www.irs.gov/irb/2005-33_IRB",
    "https://www.irs.gov/irb/2017-46_IRB",
    "https://www.irs.gov/irb/2013-52_IRB",
    "https://www.irs.gov/irb/2012-30_IRB",
    "https://www.irs.gov/irb/2010-10_IRB",
    "https://www.irs.gov/irb/2015-47_IRB",
    "https://www.irs.gov/irb/2014-33_IRB",
    "https://www.irs.gov/irb/2018-28_IRB",
    "https://www.irs.gov/irb/2022-37_IRB",
    "https://www.irs.gov/irb/2008-35_IRB",
    "https://www.irs.gov/irb/2020-26_IRB",
    "https://www.irs.gov/irb/2019-50_IRB",
    "https://www.irs.gov/irb/2015-15_IRB",
    "https://www.irs.gov/irb/2005-41_IRB",
    "https://www.irs.gov/irb/2009-27_IRB",
    "https://www.irs.gov/irb/2019-14_IRB",
    "https://www.irs.gov/irb/2004-27_IRB",
    "https://www.irs.gov/irb/2017-22_IRB",
    "https://www.irs.gov/irb/2016-33_IRB",
    "https://www.irs.gov/irb/2019-07_IRB",
    "https://www.irs.gov/irb/2009-35_IRB",
    "https://www.irs.gov/irb/2007-08_IRB",
    "https://www.irs.gov/irb/2022-49_IRB",
    "https://www.irs.gov/irb/2017-48_IRB",
    "https://www.irs.gov/irb/2021-09_IRB",
    "https://www.irs.gov/irb/2018-36_IRB",
    "https://www.irs.gov/irb/2011-42_IRB",
    "https://www.irs.gov/irb/2016-15_IRB",
    "https://www.irs.gov/irb/2003-50_IRB",
    "https://www.irs.gov/irb/2015-01_IRB",
    "https://www.irs.gov/irb/2019-12_IRB",
    "https://www.irs.gov/irb/2011-36_IRB",
    "https://www.irs.gov/irb/2019-06_IRB",
    "https://www.irs.gov/irb/2015-39_IRB",
    "https://www.irs.gov/irb/2022-52_IRB",
    "https://www.irs.gov/irb/2018-39_IRB",
    "https://www.irs.gov/irb/2009-10_IRB",
    "https://www.irs.gov/irb/2004-26_IRB",
    "https://www.irs.gov/irb/2022-22_IRB",
    "https://www.irs.gov/irb/2016-08_IRB",
    "https://www.irs.gov/irb/2013-18_IRB",
    "https://www.irs.gov/irb/2007-37_IRB",
    "https://www.irs.gov/irb/2020-23_IRB",
    "https://www.irs.gov/irb/2007-17_IRB",
    "https://www.irs.gov/irb/2023-09_IRB",
    "https://www.irs.gov/irb/2012-26_IRB",
    "https://www.irs.gov/irb/2007-20_IRB",
    "https://www.irs.gov/irb/2021-48_IRB",
    "https://www.irs.gov/irb/2003-46_IRB",
    "https://www.irs.gov/irb/2021-01_IRB",
    "https://www.irs.gov/irb/2019-04_IRB",
    "https://www.irs.gov/irb/2023-08_IRB",
    "https://www.irs.gov/irb/2015-45_IRB",
    "https://www.irs.gov/irb/2007-14_IRB",
    "https://www.irs.gov/irb/2010-13_IRB",
    "https://www.irs.gov/irb/2021-28_IRB",
    "https://www.irs.gov/irb/2023-01_IRB",
    "https://www.irs.gov/irb/2004-22_IRB",
    "https://www.irs.gov/irb/2012-33_IRB",
    "https://www.irs.gov/irb/2014-42_IRB",
    "https://www.irs.gov/irb/2004-03_IRB",
    "https://www.irs.gov/irb/2022-23_IRB",
    "https://www.irs.gov/irb/2007-04_IRB",
    "https://www.irs.gov/irb/2014-30_IRB",
    "https://www.irs.gov/irb/2003-34_IRB",
    "https://www.irs.gov/irb/2005-04_IRB",
    "https://www.irs.gov/irb/2005-30_IRB",
    "https://www.irs.gov/irb/2008-16_IRB",
    "https://www.irs.gov/irb/2020-05_IRB",
    "https://www.irs.gov/irb/2009-08_IRB",
    "https://www.irs.gov/irb/2004-04_IRB",
    "https://www.irs.gov/irb/2020-21_IRB",
    "https://www.irs.gov/irb/2016-35_IRB",
    "https://www.irs.gov/irb/2016-25_IRB",
    "https://www.irs.gov/irb/2014-17_IRB",
    "https://www.irs.gov/irb/2017-02_IRB",
    "https://www.irs.gov/irb/2013-28_IRB",
    "https://www.irs.gov/irb/2008-18_IRB",
    "https://www.irs.gov/irb/2008-46_IRB",
    "https://www.irs.gov/irb/2011-19_IRB",
    "https://www.irs.gov/irb/2017-12_IRB",
    "https://www.irs.gov/irb/2006-17_IRB",
    "https://www.irs.gov/irb/2005-47_IRB",
    "https://www.irs.gov/irb/2019-39_IRB",
    "https://www.irs.gov/irb/2017-45_IRB",
    "https://www.irs.gov/irb/2014-34_IRB",
    "https://www.irs.gov/irb/2007-29_IRB",
    "https://www.irs.gov/irb/2019-09_IRB",
    "https://www.irs.gov/irb/2016-50_IRB",
    "https://www.irs.gov/irb/2020-25_IRB",
    "https://www.irs.gov/irb/2013-45_IRB",
    "https://www.irs.gov/irb/2013-17_IRB",
    "https://www.irs.gov/irb/2008-10_IRB",
    "https://www.irs.gov/irb/2008-43_IRB",
    "https://www.irs.gov/irb/2004-09_IRB",
    "https://www.irs.gov/irb/2017-30_IRB",
    "https://www.irs.gov/irb/2014-04_IRB",
    "https://www.irs.gov/irb/2006-42_IRB",
    "https://www.irs.gov/irb/2005-11_IRB",
    "https://www.irs.gov/irb/2022-16_IRB",
    "https://www.irs.gov/irb/2009-31_IRB",
    "https://www.irs.gov/irb/2018-45_IRB",
    "https://www.irs.gov/irb/2022-51_IRB",
    "https://www.irs.gov/irb/2015-10_IRB",
    "https://www.irs.gov/irb/2005-37_IRB",
    "https://www.irs.gov/irb/2017-42_IRB",
    "https://www.irs.gov/irb/2005-14_IRB",
    "https://www.irs.gov/irb/2013-51_IRB",
    "https://www.irs.gov/irb/2011-13_IRB",
    "https://www.irs.gov/irb/2014-36_IRB",
    "https://www.irs.gov/irb/2020-51_IRB",
    "https://www.irs.gov/irb/2007-32_IRB",
    "https://www.irs.gov/irb/2010-44_IRB",
    "https://www.irs.gov/irb/2012-51_IRB",
    "https://www.irs.gov/irb/2017-24_IRB",
    "https://www.irs.gov/irb/2007-22_IRB",
    "https://www.irs.gov/irb/2006-43_IRB",
    "https://www.irs.gov/irb/2018-30_IRB",
    "https://www.irs.gov/irb/2004-20_IRB",
    "https://www.irs.gov/irb/2004-29_IRB",
    "https://www.irs.gov/irb/2006-30_IRB",
    "https://www.irs.gov/irb/2004-01_IRB",
    "https://www.irs.gov/irb/2011-41_IRB",
    "https://www.irs.gov/irb/2016-04_IRB",
    "https://www.irs.gov/irb/2010-31_IRB",
    "https://www.irs.gov/irb/2005-07_IRB",
    "https://www.irs.gov/irb/2019-23_IRB",
    "https://www.irs.gov/irb/2004-08_IRB",
    "https://www.irs.gov/irb/2018-20_IRB",
    "https://www.irs.gov/irb/2013-25_IRB",
    "https://www.irs.gov/irb/2010-46_IRB",
    "https://www.irs.gov/irb/2004-21_IRB",
    "https://www.irs.gov/irb/2014-03_IRB",
    "https://www.irs.gov/irb/2009-06_IRB",
    "https://www.irs.gov/irb/2012-16_IRB",
    "https://www.irs.gov/irb/2012-11_IRB",
    "https://www.irs.gov/irb/2006-36_IRB",
    "https://www.irs.gov/irb/2020-07_IRB",
    "https://www.irs.gov/irb/2013-26_IRB",
    "https://www.irs.gov/irb/2010-52_IRB",
    "https://www.irs.gov/irb/2006-10_IRB",
    "https://www.irs.gov/irb/2003-45_IRB",
    "https://www.irs.gov/irb/2018-40_IRB",
    "https://www.irs.gov/irb/2017-13_IRB",
    "https://www.irs.gov/irb/2010-08_IRB",
    "https://www.irs.gov/irb/2010-22_IRB",
    "https://www.irs.gov/irb/2012-13_IRB",
    "https://www.irs.gov/irb/2014-16_IRB",
    "https://www.irs.gov/irb/2014-10-IRB",
    "https://www.irs.gov/irb/2009-20_IRB",
    "https://www.irs.gov/irb/2009-24_IRB",
    "https://www.irs.gov/irb/2022-17_IRB",
    "https://www.irs.gov/irb/2011-50_IRB",
    "https://www.irs.gov/irb/2010-28_IRB",
    "https://www.irs.gov/irb/2006-26_IRB",
    "https://www.irs.gov/irb/2008-27_IRB",
    "https://www.irs.gov/irb/2018-33_IRB",
    "https://www.irs.gov/irb/2019-29_IRB",
    "https://www.irs.gov/irb/2013-36_IRB",
    "https://www.irs.gov/irb/2020-14_IRB",
    "https://www.irs.gov/irb/2019-16_IRB",
    "https://www.irs.gov/irb/2012-21_IRB",
    "https://www.irs.gov/irb/2007-15_IRB",
    "https://www.irs.gov/irb/2017-10_IRB",
    "https://www.irs.gov/irb/2008-23_irb",
    "https://www.irs.gov/irb/2022-07_IRB",
    "https://www.irs.gov/irb/2013-34_IRB",
    "https://www.irs.gov/irb/2018-27_IRB",
    "https://www.irs.gov/irb/2020-11_IRB",
    "https://www.irs.gov/irb/2005-20_IRB",
    "https://www.irs.gov/irb/2020-36_IRB",
    "https://www.irs.gov/irb/2020-44_IRB",
    "https://www.irs.gov/irb/2017-26_IRB",
    "https://www.irs.gov/irb/2018-11_IRB",
    "https://www.irs.gov/irb/2016-01_IRB",
    "https://www.irs.gov/irb/2003-51_IRB",
    "https://www.irs.gov/irb/2004-32_IRB",
    "https://www.irs.gov/irb/2005-08_IRB",
    "https://www.irs.gov/irb/2023-02_IRB",
    "https://www.irs.gov/irb/2017-39_IRB",
    "https://www.irs.gov/irb/2007-10_IRB",
    "https://www.irs.gov/irb/2012-07_IRB",
    "https://www.irs.gov/irb/2010-37_IRB",
    "https://www.irs.gov/irb/2015-27_IRB",
    "https://www.irs.gov/irb/2021-35_IRB",
    "https://www.irs.gov/irb/2021-06_IRB",
    "https://www.irs.gov/irb/2007-25_IRB",
    "https://www.irs.gov/irb/2021-05_IRB",
    "https://www.irs.gov/irb/2007-44_IRB",
    "https://www.irs.gov/irb/2006-13_IRB",
    "https://www.irs.gov/irb/2014-05_IRB",
    "https://www.irs.gov/irb/2011-43_IRB",
    "https://www.irs.gov/irb/2011-22_IRB",
    "https://www.irs.gov/irb/2016-23_IRB",
    "https://www.irs.gov/irb/2016-27_IRB",
    "https://www.irs.gov/irb/2015-14_IRB",
    "https://www.irs.gov/irb/2021-50_IRB",
    "https://www.irs.gov/irb/2012-18_IRB",
    "https://www.irs.gov/irb/2019-51_IRB",
    "https://www.irs.gov/irb/2013-01_IRB",
    "https://www.irs.gov/irb/2006-24_IRB",
    "https://www.irs.gov/irb/2018-02_IRB",
    "https://www.irs.gov/irb/2004-07_IRB",
    "https://www.irs.gov/irb/2003-40_IRB",
    "https://www.irs.gov/irb/2017-36_IRB",
    "https://www.irs.gov/irb/2013-50_IRB",
    "https://www.irs.gov/irb/2010-49_IRB",
    "https://www.irs.gov/irb/2010-35_IRB",
    "https://www.irs.gov/irb/2019-27_IRB",
    "https://www.irs.gov/irb/2009-42_IRB",
    "https://www.irs.gov/irb/2008-17_IRB",
    "https://www.irs.gov/irb/2016-38_IRB",
    "https://www.irs.gov/irb/2004-11_IRB",
    "https://www.irs.gov/irb/2020-33_IRB",
    "https://www.irs.gov/irb/2003-48_IRB",
    "https://www.irs.gov/irb/2006-48_IRB",
    "https://www.irs.gov/irb/2020-35_IRB",
    "https://www.irs.gov/irb/2006-37_IRB",
    "https://www.irs.gov/irb/2013-11_IRB",
    "https://www.irs.gov/irb/2022-08_IRB",
    "https://www.irs.gov/irb/2012-47_IRB",
    "https://www.irs.gov/irb/2014-32_IRB",
    "https://www.irs.gov/irb/2021-41_IRB",
    "https://www.irs.gov/irb/2006-20_IRB",
    "https://www.irs.gov/irb/2020-45_IRB",
    "https://www.irs.gov/irb/2023-05_IRB",
    "https://www.irs.gov/irb/2008-13_IRB",
    "https://www.irs.gov/irb/2007-41_IRB",
    "https://www.irs.gov/irb/2004-52_IRB",
    "https://www.irs.gov/irb/2017-05_IRB",
    "https://www.irs.gov/irb/2009-13_IRB",
    "https://www.irs.gov/irb/2011-15_IRB",
    "https://www.irs.gov/irb/2022-48_IRB",
    "https://www.irs.gov/irb/2011-40_IRB",
    "https://www.irs.gov/irb/2016-09_IRB",
    "https://www.irs.gov/irb/2009-04_IRB",
    "https://www.irs.gov/irb/2016-43_IRB",
    "https://www.irs.gov/irb/2022-42_IRB",
    "https://www.irs.gov/irb/2018-19_IRB",
    "https://www.irs.gov/irb/2003-35_IRB",
    "https://www.irs.gov/irb/2011-49_IRB",
    "https://www.irs.gov/irb/2006-39_IRB",
    "https://www.irs.gov/irb/2014-25_IRB",
    "https://www.irs.gov/irb/2020-15_IRB",
    "https://www.irs.gov/irb/2021-32_IRB",
    "https://www.irs.gov/irb/2016-19_IRB",
    "https://www.irs.gov/irb/2007-19_IRB",
    "https://www.irs.gov/irb/2016-24_IRB",
    "https://www.irs.gov/irb/2017-09_IRB",
    "https://www.irs.gov/irb/2006-28_IRB",
    "https://www.irs.gov/irb/2018-31_IRB",
    "https://www.irs.gov/irb/2011-25_IRB",
    "https://www.irs.gov/irb/2008-26_IRB",
    "https://www.irs.gov/irb/2004-46_IRB",
    "https://www.irs.gov/irb/2009-49_IRB",
    "https://www.irs.gov/irb/2019-36_IRB",
    "https://www.irs.gov/irb/2012-20_IRB",
    "https://www.irs.gov/irb/2018-05_IRB",
    "https://www.irs.gov/irb/2014-11_IRB",
    "https://www.irs.gov/irb/2022-43_IRB",
    "https://www.irs.gov/irb/2003-47_IRB",
    "https://www.irs.gov/irb/2015-05_IRB",
    "https://www.irs.gov/irb/2015-36_IRB",
    "https://www.irs.gov/irb/2012-23_IRB",
    "https://www.irs.gov/irb/2018-48_IRB",
    "https://www.irs.gov/irb/2022-14_IRB",
    "https://www.irs.gov/irb/2018-04_IRB",
    "https://www.irs.gov/irb/2015-23_IRB",
    "https://www.irs.gov/irb/2018-23_IRB",
    "https://www.irs.gov/irb/2011-07_IRB",
    "https://www.irs.gov/irb/2019-41_IRB",
    "https://www.irs.gov/irb/2003-38_IRB",
    "https://www.irs.gov/irb/2011-39_IRB",
    "https://www.irs.gov/irb/2012-39_IRB",
    "https://www.irs.gov/irb/2009-40_IRB",
    "https://www.irs.gov/irb/2022-10_IRB",
    "https://www.irs.gov/irb/2019-52_IRB",
    "https://www.irs.gov/irb/2006-12_IRB",
    "https://www.irs.gov/irb/2006-45_IRB",
    "https://www.irs.gov/irb/2014-43_IRB",
    "https://www.irs.gov/irb/2016-02_IRB",
    "https://www.irs.gov/irb/2011-30_IRB",
    "https://www.irs.gov/irb/2009-12_IRB",
    "https://www.irs.gov/irb/2005-18_IRB",
    "https://www.irs.gov/irb/2013-20_IRB",
    "https://www.irs.gov/irb/2016-44_IRB",
    "https://www.irs.gov/irb/2015-16_IRB",
    "https://www.irs.gov/irb/2014-31_IRB",
    "https://www.irs.gov/irb/2017-43_IRB",
    "https://www.irs.gov/irb/2016-51_IRB",
    "https://www.irs.gov/irb/2010-50_IRB",
    "https://www.irs.gov/irb/2013-10_IRB",
    "https://www.irs.gov/irb/2009-34_IRB",
    "https://www.irs.gov/irb/2006-08_IRB",
    "https://www.irs.gov/irb/2022-40_IRB",
    "https://www.irs.gov/irb/2022-09_IRB",
    "https://www.irs.gov/irb/2014-23_IRB",
    "https://www.irs.gov/irb/2013-12_IRB",
    "https://www.irs.gov/irb/2003-49_IRB",
    "https://www.irs.gov/irb/2013-21_IRB",
    "https://www.irs.gov/irb/2007-07_IRB",
    "https://www.irs.gov/irb/2020-46_IRB",
    "https://www.irs.gov/irb/2023-06_IRB",
    "https://www.irs.gov/irb/2017-29_IRB",
    "https://www.irs.gov/irb/2019-30_IRB",
    "https://www.irs.gov/irb/2016-49_IRB",
    "https://www.irs.gov/irb/2018-41_IRB",
    "https://www.irs.gov/irb/2016-17_IRB",
    "https://www.irs.gov/irb/2012-06_IRB",
    "https://www.irs.gov/irb/2005-09_IRB",
    "https://www.irs.gov/irb/2018-37_IRB",
    "https://www.irs.gov/irb/2022-27_IRB",
    "https://www.irs.gov/irb/2021-17_IRB",
    "https://www.irs.gov/irb/2006-40_IRB",
    "https://www.irs.gov/irb/2012-46_IRB",
    "https://www.irs.gov/irb/2009-18_IRB",
    "https://www.irs.gov/irb/2021-08_IRB",
    "https://www.irs.gov/irb/2020-47_IRB",
    "https://www.irs.gov/irb/2022-35_IRB",
    "https://www.irs.gov/irb/2021-42_IRB",
    "https://www.irs.gov/irb/2009-22_IRB",
    "https://www.irs.gov/irb/2007-36_IRB",
    "https://www.irs.gov/irb/2018-51_IRB",
    "https://www.irs.gov/irb/2010-24_IRB",
    "https://www.irs.gov/irb/2004-36_IRB",
    "https://www.irs.gov/irb/2019-37_IRB",
    "https://www.irs.gov/irb/2016-11_IRB",
    "https://www.irs.gov/irb/2013-03_IRB",
    "https://www.irs.gov/irb/2007-43_IRB",
    "https://www.irs.gov/irb/2016-30_IRB",
    "https://www.irs.gov/irb/2014-53_IRB",
    "https://www.irs.gov/irb/2010-03_IRB",
    "https://www.irs.gov/irb/2018-42_IRB",
    "https://www.irs.gov/irb/2014-27_IRB",
    "https://www.irs.gov/irb/2010-34_IRB",
    "https://www.irs.gov/irb/2008-22_IRB",
    "https://www.irs.gov/irb/2020-32_IRB",
    "https://www.irs.gov/irb/2020-10_IRB",
    "https://www.irs.gov/irb/2013-15_IRB",
    "https://www.irs.gov/irb/2006-34_IRB",
    "https://www.irs.gov/irb/2022-24_IRB",
    "https://www.irs.gov/irb/2020-19_IRB",
    "https://www.irs.gov/irb/2019-44_IRB",
    "https://www.irs.gov/irb/2009-46_IRB",
    "https://www.irs.gov/irb/2011-33_IRB",
    "https://www.irs.gov/irb/2004-38_IRB",
    "https://www.irs.gov/irb/2022-02_IRB",
    "https://www.irs.gov/irb/2019-08_IRB",
    "https://www.irs.gov/irb/2007-12_IRB",
    "https://www.irs.gov/irb/2010-01_IRB",
    "https://www.irs.gov/irb/2008-06_IRB",
    "https://www.irs.gov/irb/2010-43_IRB",
    "https://www.irs.gov/irb/2008-28_IRB",
    "https://www.irs.gov/irb/2012-38_IRB",
    "https://www.irs.gov/irb/2018-29_IRB",
    "https://www.irs.gov/irb/2006-15_IRB",
    "https://www.irs.gov/irb/2004-33_IRB",
    "https://www.irs.gov/irb/2010-05_IRB",
    "https://www.irs.gov/irb/2020-02_IRB",
    "https://www.irs.gov/irb/2017-44_IRB",
    "https://www.irs.gov/irb/2005-25_IRB",
    "https://www.irs.gov/irb/2009-52_IRB",
    "https://www.irs.gov/irb/2008-25_IRB",
    "https://www.irs.gov/irb/2017-50_IRB",
    "https://www.irs.gov/irb/2003-37_IRB",
    "https://www.irs.gov/irb/2020-20_IRB",
    "https://www.irs.gov/irb/2019-10_IRB",
    "https://www.irs.gov/irb/2012-49_IRB",
    "https://www.irs.gov/irb/2020-42_IRB",
    "https://www.irs.gov/irb/2021-30_IRB",
    "https://www.irs.gov/irb/2007-09_IRB",
    "https://www.irs.gov/irb/2021-45_IRB",
    "https://www.irs.gov/irb/2009-44_IRB",
    "https://www.irs.gov/irb/2008-40_IRB",
    "https://www.irs.gov/irb/2004-18_IRB",
    "https://www.irs.gov/irb/2010-51_IRB",
    "https://www.irs.gov/irb/2010-07_IRB",
    "https://www.irs.gov/irb/2008-50_IRB",
    "https://www.irs.gov/irb/2003-27_IRB",
    "https://www.irs.gov/irb/2021-39_IRB",
    "https://www.irs.gov/irb/2015-50_IRB",
    "https://www.irs.gov/irb/2014-21_IRB",
    "https://www.irs.gov/irb/2010-02_IRB",
    "https://www.irs.gov/irb/2008-21_IRB",
    "https://www.irs.gov/irb/2007-35_IRB",
    "https://www.irs.gov/irb/2013-33_IRB",
    "https://www.irs.gov/irb/2015-04_IRB",
    "https://www.irs.gov/irb/2018-15_IRB",
    "https://www.irs.gov/irb/2014-02_IRB",
    "https://www.irs.gov/irb/2021-47_IRB",
    "https://www.irs.gov/irb/2019-48_IRB",
    "https://www.irs.gov/irb/2014-44_IRB",
    "https://www.irs.gov/irb/2010-32_IRB",
    "https://www.irs.gov/irb/2015-06_IRB",
    "https://www.irs.gov/irb/2021-07_IRB",
    "https://www.irs.gov/irb/2014-08_IRB",
    "https://www.irs.gov/irb/2017-47_IRB",
    "https://www.irs.gov/irb/2009-47_IRB",
    "https://www.irs.gov/irb/2023-03_IRB",
    "https://www.irs.gov/irb/2021-31_IRB",
    "https://www.irs.gov/irb/2018-43_IRB",
    "https://www.irs.gov/irb/2009-15_IRB",
    "https://www.irs.gov/irb/2015-20_IRB",
    "https://www.irs.gov/irb/2009-37_IRB",
    "https://www.irs.gov/irb/2003-52_IRB",
    "https://www.irs.gov/irb/2014-48_IRB",
    "https://www.irs.gov/irb/2013-22_IRB",
    "https://www.irs.gov/irb/2007-13_IRB",
    "https://www.irs.gov/irb/2019-03_IRB",
    "https://www.irs.gov/irb/2006-35_IRB",
    "https://www.irs.gov/irb/2011-21_IRB",
    "https://www.irs.gov/irb/2016-07_IRB",
    "https://www.irs.gov/irb/2009-39_IRB",
    "https://www.irs.gov/irb/2008-15_IRB",
    "https://www.irs.gov/irb/2010-42_IRB",
    "https://www.irs.gov/irb/2005-35_IRB",
    "https://www.irs.gov/irb/2018-46_IRB",
    "https://www.irs.gov/irb/2014-46_IRB",
    "https://www.irs.gov/irb/2008-37_IRB",
    "https://www.irs.gov/irb/2015-11_IRB",
    "https://www.irs.gov/irb/2008-48_IRB",
    "https://www.irs.gov/irb/2006-21_IRB",
    "https://www.irs.gov/irb/2009-21_IRB",
    "https://www.irs.gov/irb/2008-36_IRB",
    "https://www.irs.gov/irb/2010-48_IRB",
    "https://www.irs.gov/irb/2010-30_IRB",
    "https://www.irs.gov/irb/2020-30_IRB",
    "https://www.irs.gov/irb/2021-16_IRB",
    "https://www.irs.gov/irb/2015-25_IRB",
    "https://www.irs.gov/irb/2020-38_IRB",
    "https://www.irs.gov/irb/2021-33_irb",
    "https://www.irs.gov/irb/2021-18_IRB",
    "https://www.irs.gov/irb/2012-02_IRB",
    "https://www.irs.gov/irb/2007-03_IRB",
    "https://www.irs.gov/irb/2014-26_IRB",
    "https://www.irs.gov/irb/2004-40_IRB",
    "https://www.irs.gov/irb/2018-35_IRB",
    "https://www.irs.gov/irb/2016-47_IRB",
    "https://www.irs.gov/irb/2005-27_IRB",
    "https://www.irs.gov/irb/2021-40_IRB",
    "https://www.irs.gov/irb/2019-22_IRB",
    "https://www.irs.gov/irb/2019-31_IRB",
    "https://www.irs.gov/irb/2013-06_IRB",
    "https://www.irs.gov/irb/2006-07_IRB",
    "https://www.irs.gov/irb/2017-49_IRB",
    "https://www.irs.gov/irb/2006-01_IRB",
    "https://www.irs.gov/irb/2020-29_IRB",
    "https://www.irs.gov/irb/2022-15_IRB",
    "https://www.irs.gov/irb/2017-07_IRB",
    "https://www.irs.gov/irb/2011-23_IRB",
    "https://www.irs.gov/irb/2019-35_IRB",
    "https://www.irs.gov/irb/2013-19_IRB",
    "https://www.irs.gov/irb/2019-32_IRB",
    "https://www.irs.gov/irb/2019-43_IRB",
    "https://www.irs.gov/irb/2012-29_IRB",
    "https://www.irs.gov/irb/2004-16_IRB",
    "https://www.irs.gov/irb/2022-33_IRB",
    "https://www.irs.gov/irb/2012-32_IRB",
    "https://www.irs.gov/irb/2016-31_IRB",
    "https://www.irs.gov/irb/2021-19_IRB",
    "https://www.irs.gov/irb/2018-12_IRB",
    "https://www.irs.gov/irb/2006-19_IRB",
    "https://www.irs.gov/irb/2006-51_IRB",
    "https://www.irs.gov/irb/2011-04_IRB",
    "https://www.irs.gov/irb/2008-51_IRB",
    "https://www.irs.gov/irb/2022-50_IRB",
    "https://www.irs.gov/irb/2015-17_IRB",
    "https://www.irs.gov/irb/2021-38_IRB",
    "https://www.irs.gov/irb/2013-47_IRB",
    "https://www.irs.gov/irb/2010-17_IRB",
    "https://www.irs.gov/irb/2011-27_IRB",
    "https://www.irs.gov/irb/2010-38_IRB",
    "https://www.irs.gov/irb/2013-46_IRB",
    "https://www.irs.gov/irb/2016-42_IRB",
    "https://www.irs.gov/irb/2005-42_IRB",
    "https://www.irs.gov/irb/2012-35_IRB",
    "https://www.irs.gov/irb/2019-38_IRB",
    "https://www.irs.gov/irb/2004-19_IRB",
    "https://www.irs.gov/irb/2014-52_IRB",
    "https://www.irs.gov/irb/2015-09_IRB",
    "https://www.irs.gov/irb/2007-31_IRB",
    "https://www.irs.gov/irb/2015-33_IRB",
    "https://www.irs.gov/irb/2019-18_IRB",
    "https://www.irs.gov/irb/2017-25_IRB",
    "https://www.irs.gov/irb/2005-26_IRB",
    "https://www.irs.gov/irb/2015-24_IRB",
    "https://www.irs.gov/irb/2013-14_IRB",
    "https://www.irs.gov/irb/2008-33_IRB",
    "https://www.irs.gov/irb/2010-29_IRB",
    "https://www.irs.gov/irb/2010-26_IRB",
    "https://www.irs.gov/irb/2016-26_IRB",
    "https://www.irs.gov/irb/2005-34_IRB",
    "https://www.irs.gov/irb/2004-05_IRB",
    "https://www.irs.gov/irb/2010-47_IRB",
    "https://www.irs.gov/irb/2010-14_IRB",
    "https://www.irs.gov/irb/2017-06_IRB",
    "https://www.irs.gov/irb/2004-10_IRB",
    "https://www.irs.gov/irb/2012-22_IRB",
    "https://www.irs.gov/irb/2011-28_IRB",
    "https://www.irs.gov/irb/2007-21_IRB",
    "https://www.irs.gov/irb/2010-23_IRB",
    "https://www.irs.gov/irb/2022-26_IRB",
    "https://www.irs.gov/irb/2015-21_IRB",
    "https://www.irs.gov/irb/2014-49_IRB",
    "https://www.irs.gov/irb/2004-12_IRB",
    "https://www.irs.gov/irb/2005-13_IRB",
    "https://www.irs.gov/irb/2007-18_IRB",
    "https://www.irs.gov/irb/2009-03_IRB",
    "https://www.irs.gov/irb/2017-03_IRB",
    "https://www.irs.gov/irb/2009-50_IRB",
    "https://www.irs.gov/irb/2006-47_IRB",
    "https://www.irs.gov/irb/2019-20_IRB",
    "https://www.irs.gov/irb/2022-06_IRB",
    "https://www.irs.gov/irb/2003-31_IRB",
    "https://www.irs.gov/irb/2014-07_IRB",
    "https://www.irs.gov/irb/2010-20_IRB",
    "https://www.irs.gov/irb/2019-26_IRB",
    "https://www.irs.gov/irb/2012-43_IRB",
    "https://www.irs.gov/irb/2005-22_IRB",
    "https://www.irs.gov/irb/2005-19_IRB",
    "https://www.irs.gov/irb/2020-17_IRB",
    "https://www.irs.gov/irb/2016-12_IRB",
    "https://www.irs.gov/irb/2013-39_IRB",
    "https://www.irs.gov/irb/2012-40_IRB",
    "https://www.irs.gov/irb/2016-32_IRB",
    "https://www.irs.gov/irb/2008-09_IRB",
    "https://www.irs.gov/irb/2009-05_IRB",
    "https://www.irs.gov/irb/2010-39_IRB",
    "https://www.irs.gov/irb/2005-43_IRB",
    "https://www.irs.gov/irb/2008-39_IRB",
    "https://www.irs.gov/irb/2012-15_IRB",
    "https://www.irs.gov/irb/2010-09_IRB",
    "https://www.irs.gov/irb/2010-33_IRB",
    "https://www.irs.gov/irb/2017-18_IRB",
    "https://www.irs.gov/irb/2007-50_IRB",
    "https://www.irs.gov/irb/2019-40_IRB"
]


Load Internal Revenue Bulletins

In [ ]:
loader = UnstructuredURLLoader(urls=urls)

In [ ]:
irb_data = loader.load()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
collection_name="irb"
persist_directory="/content/drive/My Drive/Colab Notebooks/chromadb/irb"

In [ ]:
text_splitter = TokenTextSplitter(chunk_size=1000, chunk_overlap=0)
irb_doc = text_splitter.split_documents(irb_data)

In [ ]:
embeddings = OpenAIEmbeddings()
irb_db = Chroma.from_documents(irb_doc, embeddings, collection_name=collection_name, persist_directory=persist_directory)
irb_db.persist()

Running Chroma using direct local API.


FloatProgress(value=0.0, layout=Layout(width='100%'), style=ProgressStyle(bar_color='black'))

loaded in 58563 embeddings
loaded in 1 collections
collection with name irb already exists, returning existing collection


DEBUG:Chroma:Index saved to /content/drive/My Drive/Colab Notebooks/chromadb/irb/index/index.bin


Persisting DB to disk, putting it in the save folder /content/drive/My Drive/Colab Notebooks/chromadb/irb
PersistentDuckDB del, about to run persist
Persisting DB to disk, putting it in the save folder /content/drive/My Drive/Colab Notebooks/chromadb/irb


FloatProgress(value=0.0, layout=Layout(width='100%'), style=ProgressStyle(bar_color='black'))